In [1]:
#supress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Data Reading and Inspection

In [ ]:
import os
#get current working directory
cwd = os.getcwd()
print(cwd)

#ASSUMPTION - All the DatasetI-B files are in NAD folder inside the working directory
#listing down the files in the NAD folder and get the State names in a separate 
lt=os.listdir(cwd+"\\NAD\\")
StatesList=[]
import re
for i in range(0, len(lt)):
    StatesList.append((re.findall(r'-\w+?-',lt[i])[0][1:-1]).replace("_"," "))
StatesList

In [ ]:
#reading datasetI-B
lt=os.listdir(cwd+"\\NAD\\")
NADDir=cwd+"\\NAD\\"
newlt=[]

for i in range(0 , len(lt)):
    currentfile=pd.read_csv(NADDir+lt[i],encoding = "latin1" )
    currentfile['State']=StatesList[i]
    newlt.append(currentfile)

DF_LT= pd.concat(newlt,axis=0,ignore_index=True)

In [ ]:
#reading data-setI-A
std=cwd+"\\State-wise Gross Domestic Product (GDP) at current Price on yearly basis.csv"
dataset1 = pd.read_csv(std)
dataset1

In [ ]:
#inspecting shape
dataset1.shape

In [ ]:
#inspecting dataframe
dataset1.info()

In [ ]:
#data variance across columns
dataset1.describe()

In [ ]:
#inspecting null values across columns
dataset1.isnull().sum()

In [ ]:
#inspecting null values across rows
dataset1.isnull().sum(axis=1)

### Part-I: GDP Analysis of Indian States
For each of the following steps of analysis, choose an appropriate type of plot for comparing the data. Also, ensure that the plots are in increasing or decreasing order for better comparison. For e.g., if you make a bar plot to compare the GDPs of various states, ensure that the bars are in either increasing or decreasing order of GDP.
 
# Part I-A:
- For the analysis below, use the Data I-A.
- Remove the rows: '(% Growth over the previous year)' and 'GSDP - CURRENT PRICES (in Crore) for the year 2016-17.
- Calculate the average growth of states over the duration 2013-14, 2014-15 and 2015-16 by taking the mean of the row '(% Growth over previous year)'. Compare the calculated value and plot it for the states. Make appropriate transformations if necessary to plot the data. Report the average growth rates of the various states:
        - Which states have been growing consistently fast, and which ones have been struggling?
        - Curiosity exercise - what has been the average growth rate of your home state, and how does it compare to the national average over this duration?
- Plot the total GDP of the states for the year 2015-16:
- Identify the top-5 and the bottom-5 states based on total GDP

In [ ]:
#clearly WestBengal has no significance in the dataset as all the values are null, thus removing it
dataset1=dataset1.drop(['West Bengal1'] , axis=1)

In [ ]:
#Part1-A
# removing row  'GSDP - CURRENT PRICES (in Crore)'  for 2016-17
dataset1=dataset1.drop(dataset1.index[5])

In [ ]:
#Part1-A
# removing row  '(% Growth over the previous year)' for 2016-17
dataset1=dataset1.drop(dataset1.index[9]).reset_index(drop=True) 
dataset1

In [ ]:
#creating data-set for % Growth over previous year
Growth_Per_Df=dataset1[(dataset1.iloc[:,0]=='(% Growth over previous year)') ]
Growth_Per_Df

In [ ]:
Growth_Per_Df=Growth_Per_Df.fillna(Growth_Per_Df.mean())

**Calculate the average growth of states over the duration 2013-14, 2014-15 and 2015-16 by taking the mean of the row '(% Growth over previous year)'. Compare the calculated value and plot it for the states.**

In [ ]:
Growth_Per_Df_set1=Growth_Per_Df[Growth_Per_Df.Duration!='2012-13']

In [ ]:
dt=pd.DataFrame(Growth_Per_Df_set1.mean(), columns=['Values']).sort_values(by='Values',ascending=False).reset_index()
dt.columns=['State' , 'Mean_value']
dt

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x='State',y='Mean_value',data=dt,palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("average % Growth over previous year)of states over the duration 2013-14, 2014-15 and 2015-16 ")
plt.show()

In [ ]:
DF_13_14_Mean=pd.DataFrame(Growth_Per_Df_set1[ 
    (Growth_Per_Df_set1.Duration=='2013-14')].mean(), columns=['Values']).sort_values(by='Values',ascending=False).reset_index()
DF_13_14_Mean.columns=['State' , 'Mean_value']

DF_14_15_Mean=pd.DataFrame(Growth_Per_Df_set1[
    (Growth_Per_Df_set1.Duration=='2014-15')].mean(), columns=['Values']).sort_values(by='Values',ascending=False).reset_index()
DF_14_15_Mean.columns=['State' , 'Mean_value']

DF_15_16_Mean=pd.DataFrame(Growth_Per_Df_set1[ 
    (Growth_Per_Df_set1.Duration=='2015-16')].mean(), columns=['Values']).sort_values(by='Values',ascending=False).reset_index()
DF_15_16_Mean.columns=['State' , 'Mean_value']

In [ ]:
Growth_state_merged=pd.merge(DF_13_14_Mean,DF_14_15_Mean,on='State')
Growth_state_merged=pd.merge(Growth_state_merged,DF_15_16_Mean,on='State')
Growth_state_merged.columns=['State' , '2013-14' , '2014-15' , '2015-16']
Growth_state_merged

Looks like Jammu & Kashmir , Andhra Pradesh and Goa states made a significant improvement  while if we see growth rate dropped signifcantly for Mizoram and Nagaland. Meghalaya has been struggling also and so does Sikkim and Odisha.

In [ ]:
plt.figure(figsize=(31,15))
plt.subplot(1,3,1)
sns.barplot(x='State',y='Mean_value',data=DF_13_14_Mean,palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("average % Growth over previous year)of states over the duration 2013-14 ")
plt.subplot(1,3,2)
sns.barplot(x='State',y='Mean_value',data=DF_14_15_Mean,palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("average % Growth over previous year)of states over the duration 2014-15 ")
plt.subplot(1,3,3)
sns.barplot(x='State',y='Mean_value',data=DF_15_16_Mean,palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("average % Growth over previous year)of states over the duration 2015-16 ")
plt.show()

In [ ]:
#creating a data-set for GDP rows and keeping only states
Total_GDP_Df=dataset1[(dataset1.iloc[:,0]!='(% Growth over previous year)') ]
Total_GDP_Df=Total_GDP_Df.drop('All_India GDP', axis=1)

In [ ]:
#assumption is based on median value for each state, imputing nulls
Total_GDP_Df=Total_GDP_Df.fillna(Total_GDP_Df.median())

#### Plot the total GDP of the states for the year 2015-16

In [ ]:
#extracting on 2015-16 year records
Total_GDP_Df_15_16=Total_GDP_Df[(Total_GDP_Df.Duration=='2015-16')]

In [ ]:
#using melt function for unpivoting state columns and its values
Total_GDP_Df_15_16=pd.melt(Total_GDP_Df_15_16 , id_vars=Total_GDP_Df_15_16.columns[0:2],value_name='Values',var_name='State'
                          ).sort_values( by='Values' , ascending=False)
Total_GDP_Df_15_16

In [ ]:
#Total GDP for year 2015-16
plt.figure(figsize=(15,10))
sns.barplot(x='State',y='Values',data=Total_GDP_Df_15_16,palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("Total GDP for year 2015-16 ")
plt.show()

#### Top 5 State for Total GDP in Year 2015-16
- Maharashtra
- Tamil Nadu
- Uttar Pradesh
- Karnataka
- Gujarat

#### Bottom 5 State for Total GDP in Year 2015-16
- Arunachal Pradesh
- Sikkim
- Nagaland
- Manipur
- Mizoram


# Part I-B:
For the analysis below, use Data I-B. You can also use Data I-B along with Data I-A if required. Also, perform the analysis only for the duration : 2014-15. 
- Filter out the Union Territories (Delhi, Chandigarh, Andaman and Nicobar Islands etc.) for further analysis since they are governed directly by the centre, not state governments.
- Plot the GDP per capita for all the states.
- Identify the top-5 and the bottom-5 states based on GDP per capita.
- Find the ratio of highest per capita GDP to the lowest per capita GDP.
- Plot the percentage contribution of primary, secondary and tertiary sectors as a percentage of total GDP for all the states.
- Categorise the states into four categories based on GDP per capita (C1, C2, C3, C4 - C1 would have the highest per capita GDP, C4 the lowest). The quantile values are (0.20,0.5, 0.85, 1), i.e. the states lying between the 85th and the 100th percentile are in C1, those between 50th and 85th percentile are in C2 and so on.
Note: Categorisation into four categories will simplify the subsequent analysis, since comparing data of all the states would become quite exhaustive

For each category C1, C2, C3, C4:
- Find the top 3/4/5 sub-sectors (such as agriculture, forestry and fishing, crops, manufacturing etc.) [not primary, secondary and tertiary] which contribute to approx. 80% of the GSDP of each category
Note-I: The nomenclature for this project is as follows: primary, secondary and tertiary are named 'sectors', while agriculture, manufacturing etc. are named 'sub-sectors'
Note-II: If the top-3 sub-sectors contribute to say 79% of the GDP of some category, you can report 'these top-3 sub-sectors contribute to approx. 80% of the GDP'. This is to simplify the analysis and make the results consumable (remember, the CEO has to present the report to the CMs, and CMs have limited time, so the analysis needs to be very sharp and concise)
- Plot the contribution of the sub-sectors as a percentage of the GSDP of each category.  

In [ ]:
#reading from dataset I-B , dropping records for other years as we will perform analysis on 2014-15 duration only
DF14_15=DF_LT.drop(['2011-12', '2012-13', '2013-14','2015-16', '2016-17'] , axis=1)

In [ ]:
DF14_15.info()
# rename columns 
DF14_15.columns=['Values', 'Item', 'S.No.', 'State']

In [ ]:
#inspecting percentile values
DF14_15.describe(percentiles=[0.25,0.50,0.75,0.90,0.95,0.99])

In [ ]:
DF14_15.isnull().sum()

In [ ]:
#assumption - imputing null values with 0
DF14_15['Values'].fillna(0,inplace=True)

In [ ]:
#creating two separate Dataframe - One for totals of all sector and others of sectors alone
DF_Total=DF14_15[DF14_15['S.No.']=='Total']
DF_Sectors=DF14_15[~(DF14_15['S.No.']=='Total')]

In [ ]:
#creating separate data-frames for Primary , secondary and tertiary sectors
DF_Sectors['S.No.']=DF_Sectors['S.No.'].astype('float64')
PrimarySector_DF=DF_Sectors[DF_Sectors['S.No.'].isin([1,2])]
PrimarySector_DF['Sector']="Primary"
SecondarySector_DF=DF_Sectors[DF_Sectors['S.No.'].isin([3,4,5])]
SecondarySector_DF['Sector']="Secondary"
TertiarySector_DF=DF_Sectors[DF_Sectors['S.No.'].isin([6,7,8,9,10,11])]
TertiarySector_DF['Sector']="tertiary"
OverallState_DF=DF_Sectors[DF_Sectors['S.No.'].isin([12,13,14,15,16,17])]

In [ ]:
#concating them into one data-frame to use it for further analysis
d_sector=pd.concat([PrimarySector_DF,SecondarySector_DF,TertiarySector_DF],axis=0).sort_values(by='State')
d_sector.head()

In [ ]:
#computing per capita GDP for each state
PerCapitaDF=OverallState_DF[OverallState_DF.Item=='Per Capita GSDP (Rs.)'].sort_values(by='Values',ascending=False)
PerCapitaDF

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x='State',y='Values',data=PerCapitaDF,palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("Plot Per Capita GSDP (Rs.) for year 2014-15 ")
plt.show()

#### Top 5 States for per Capita GSDP in Year 2014-15
- Goa
- Sikkim
- Haryana
- Kerala
- Uttarakhand

#### Bottom 5 States for per Capita GSDP in Year 2014-15
- Jharkhand
- Assam
- Manipur
- Uttar Pradesh
- Bihar

**Find the ratio of highest to lowest**

Highest state is Goa with 271793 and lowest state is Bihar with 271793 total per capita GDP. Then their ratio would be 

In [ ]:
Ratio=list(PerCapitaDF[PerCapitaDF.State=='Goa'].Values.values/PerCapitaDF[PerCapitaDF.State=='Bihar'].Values.values)
print(round(Ratio[0],3))

**Plot the percentage contribution of primary, secondary and tertiary sectors as a percentage of total GDP for all the states.**

In [ ]:
# a new dataframe for total GSDP for a state
Newdf=OverallState_DF[OverallState_DF.Item=='Gross State Domestic Product']

In [ ]:
#using a merge query to concatenate total state GSDP and sector-wise GDP
totalgdpmerge_df=pd.merge(Newdf,DF_Total,on='State')
totalgdpmerge_df=totalgdpmerge_df.drop(['Item_x', 'S.No._x', 'S.No._y'], axis=1)

In [ ]:
#creating new attribute for calculating Percentage for each sector for each state againsts its total GSDP
totalgdpmerge_df.columns=['Total_State_GDP', 'State', 'Sector_GDP', 'Item']
totalgdpmerge_df['Percentage']=round(totalgdpmerge_df['Sector_GDP']/totalgdpmerge_df['Total_State_GDP']*100,2)
totalgdpmerge_df

## Categorize data into 4 categories
Categorise the states into four categories based on GDP per capita (C1, C2, C3, C4 - C1 would have the highest per capita GDP, C4 the lowest). The quantile values are (0.20,0.5, 0.85, 1), i.e. the states lying between the 85th and the 100th percentile are in C1, those between 50th and 85th percentile are in C2 and so on.

In [ ]:
PerCapitaDF_cat=PerCapitaDF
#calculating quantile values and defining a function to assign categories to each state based on its per capita values
upp=PerCapitaDF_cat.Values.quantile(1)
print(upp)
lw1=PerCapitaDF_cat.Values.quantile(0.85)
print(lw1)
lw2=PerCapitaDF_cat.Values.quantile(0.50)
print(lw2)
lw3=PerCapitaDF_cat.Values.quantile(0.20)
print(lw3)
lw=PerCapitaDF_cat.Values.quantile(0)
print(lw)
PerCapitaDF_cat['Category']=""
def newfunc(x):
    if ((x>lw1) & (x<=upp)):
         return "C1"
    elif ( (x>lw2) & (x<=lw1)):
        return "C2"
    elif ((x>lw3) & (x<=lw2)):
        return "C3"
    else:
        return "C4"
PerCapitaDF_cat['Category']=PerCapitaDF_cat.Values.apply(newfunc)
PerCapitaDF_cat

In [ ]:
#creating two dataframes :
## OverallState_DF_Total - for getting the GSDP along with the category for each state
## Category_sum_df - dataframe holding total values for each category
OverallState_DF_Total=pd.merge(PerCapitaDF_cat,Newdf, on='State')
Category_sum_df=pd.DataFrame(OverallState_DF_Total.groupby('Category').Values_y.sum()).reset_index()
Category_sum_df.columns=['Category','Category_total']
Category_sum_df

In [ ]:
OverallState_DF_Total=OverallState_DF_Total.drop(['Values_x', 'Item_x', 'S.No._x', 'S.No._y'] , axis=1)
OverallState_DF_Total

In [ ]:
#creating new dataframe sector_wise_df for evaluating total values against category, sector and sub-sector together
OverallState_DF_Total1=pd.merge(PerCapitaDF_cat,d_sector, on='State')
OverallState_DF_Total1=OverallState_DF_Total1.drop(['Values_x','Item_x','S.No._x'],axis=1)
sector_wise_df=OverallState_DF_Total1.groupby(['Category','Item_y','Sector']).agg({'Values_y':'sum'}).reset_index()
sector_wise_df

In [ ]:
#merging the dataframe to get the total values against each category and thus calculating percentage
master_merge=pd.merge(sector_wise_df,Category_sum_df,on='Category')
master_merge.columns=['Category', 'Sub-sector', 'Sector', 'Sub-Sector_Total', 'Category_Total']
master_merge['Percent']=round(master_merge['Sub-Sector_Total']
                                     /master_merge['Category_Total']*100,2)
master_merge=master_merge.sort_values(by=['Category','Percent'] , ascending=False)
master_merge

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='Sub-sector',y='Percent',data=master_merge[master_merge.Category=='C1'],palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("Plot Sub-Sectors of C1 Category ")
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='Sub-sector',y='Percent',data=master_merge[master_merge.Category=='C2'],palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("Plot Sub-Sectors of C2 Category ")
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='Sub-sector',y='Percent',data=master_merge[master_merge.Category=='C3'],palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("Plot Sub-Sectors of C3 Category ")
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='Sub-sector',y='Percent',data=master_merge[master_merge.Category=='C4'],palette="Blues_d")
plt.xticks(rotation='vertical')
plt.title("Plot Sub-Sectors of C4 Category ")
plt.show()

- For c1 category states, real-state sector has the highest contribution. But one can focus on transportation and  industrial areas to increase employability and thus this helps in per capita GDP.
- In all the 4 categories, public administration is one sector which generates not more than 6% of total state per capita. Thus this is also one potential area where government can focus to see how we can improve the facilities further.
- Agriculture, Forestry and fishing sub-sector is always among top 3 sectors for per-capita GDP generation. As India is known to be diversified country in terms of flora and fauna, we can focus on this sector more to generate maximum revenue and increase the number of exports!

# Part-II: GDP and Education Drop-out Rates
In Part-I, you would have noticed that (one) way to increase per capita GDP is by shifting the distribution of GDP towards the secondary and tertiary sectors, i.e. the manufacturing and services industries. But these industries can thrive only when there is availability of educated, skilled labour.
In this part of the analysis, you will investigate whether there is any relationship between per capita GDP with drop-out rates in education.
 
- Analyse if there is any correlation of GDP per capita with dropout rates in education (primary, upper primary and secondary) for the year 2014-2015 for the states. Choose an appropriate plot to conduct this analysis.
- Write the key insights you observe from this data:
- Form at least one reasonable hypothesis for the observations from the data

In [ ]:
#reading data-setI-A
std=cwd+"\\rs_session243_au570_1.1.csv"
dataset2 = pd.read_csv(std)
#dataset2.fillna(0,inplace=True)
dataset2.head()

In [ ]:
dataset2.info()

In [ ]:
#analzying for Primary and secondary education for year 2014-15 
col=['Sl. No.', 'Level of Education - State', 'Primary - 2014-2015', 'Primary - 2014-2015.1',
     'Secondary - 2014-2015']
dataset2=dataset2[col]
#replacing null values with 0
dataset2.fillna(0,inplace=True)
#adding values for primary education
dataset2['Primary - 2014-2015']=dataset2['Primary - 2014-2015']+dataset2['Primary - 2014-2015.1']
#dropping unecessary columns
dataset2=dataset2.drop(['Primary - 2014-2015.1'],axis=1)

In [ ]:
dataset2

In [ ]:
df=pd.merge(dataset2,OverallState_DF_Total,left_on='Level of Education - State',right_on='State')

In [ ]:
sns.heatmap(df.corr(),annot=True)
plt.title("Correlation between drop-out rate and Total state GDP")
plt.show()

In [ ]:
df=pd.merge(dataset2,PerCapitaDF,left_on='Level of Education - State',right_on='State')

In [ ]:
sns.heatmap(df.corr(),annot=True)
plt.title("Correlation between drop-out rate and GDP per Capita")
plt.show()

GDP per capita is negatively related to the number of drop-outs for Primary Education. This means, if the number of primary education drop-outs,it would negatively impact the per capita GDP of the state.

Education is an important sector among all and if we want to increase the employability, government should continue to focus on building base for primary and secondary school education.  